In [1]:
import pandas as pd
import requests
import numpy as np

In [3]:
query = input("Ingrese palabra a buscar en Twitter")

In [38]:
url = "https://twitter154.p.rapidapi.com/search/search"

querystring = {"query":f"#{query}","section":"top","start_date":"2022-01-01"}

headers = {
	"X-RapidAPI-Key": "c6a5a891ecmsh03d9ec66402dd34p18a3c6jsnbe65087e49e4",
	"X-RapidAPI-Host": "twitter154.p.rapidapi.com"
}

In [39]:
response = requests.request("GET", url, headers=headers, params=querystring)

In [40]:
res_dict = response.json()
res_dict = res_dict['results']

In [41]:
df = pd.DataFrame.from_dict(res_dict)

In [42]:
df

,tweet_id,creation_date,text,media_url,video_url,user,language,favorite_count,retweet_count,reply_count,quote_count,retweet,timestamp,video_view_count
0,1582073959960526876,Mon Oct 17 18:20:00 +0000 2022,America’s #ICBM force is a key part of our #nu...,None,None,{'creation_date': 'Thu Sep 16 02:46:03 +0000 2...,en,2,1,0,0,False,1666030800,None
1,1583904843147915264,Sat Oct 22 19:35:16 +0000 2022,SATURDAY DOUBLE FEATURE TIME!\n\nPatreons and ...,None,None,{'creation_date': 'Sun Apr 12 21:20:51 +0000 2...,en,0,0,0,0,False,1666467316,None
2,1582874426777174016,Wed Oct 19 23:20:46 +0000 2022,"🤯\nI GUESS THE STORM IS ☢️NUCLEAR☢️, that's wh...",None,None,{'creation_date': 'Sat Mar 28 12:06:23 +0000 2...,en,1,0,1,0,False,1666221646,None
3,1540902765324640256,Sun Jun 26 03:40:22 +0000 2022,Rs-28 Sarmat ICBM travels at a speed of MACH 2...,None,None,{'creation_date': 'Sat Apr 23 02:49:27 +0000 2...,en,968,447,102,79,False,1656214822,None
4,1580928751390842881,Fri Oct 14 14:29:21 +0000 2022,On the left is a #Russian 2 million dollar #IC...,None,None,{'creation_date': 'Sat Jul 16 16:47:20 +0000 2...,en,87,17,8,0,False,1665757761,None
5,1582725956988329984,Wed Oct 19 13:30:48 +0000 2022,18 minutes... and the entire NY City and Washi...,None,None,{'creation_date': 'Mon Apr 04 00:47:31 +0000 2...,en,60,23,10,2,False,1666186248,None
